# Parse development book

### Notebook purpose
This notebook is development space for python parse.ts replacement and upgrade.
It reads specified google sheets and output actants.json file, which can be imported to inkVisitor RethinkDB.py

The aggreated doc for dev: https://docs.google.com/document/d/1ga6R_9TWAQlXE9XqPE_qZ2S8S8aVW2A7n8SuYpSXnoI/edit#heading=h.q9ntf0ofam2u
The holy schema: https://app.diagrams.net/#G1bKvqEKr6JzPWryVg-vYudQy_4KvuzHS9

### JSon schemas for the actants
 * are created from *.ts files through typescript-jschema commandline utility
 * the schemas are trasformed as python Classes through warlock library


### Principles of the parsing operations
 * There are google sheet dataset, which need to be transformed to json format according to inkVisior datamodel
 * inkVisitor holds datamodel in the code, the typescript classes (see /shared/types...), or the holy schema
 * DZ created in input table parsing instructions
   * the first 5 rows contain instructions
   * if the keywords contain ? character, they are ignored (it is work in progress from DZ)

#### Parsing instructions
 * explicit set of keywords
   * discard
     * ...
   * inside
     * the value in the column should be straitforwadly made part of the entity object, the exists update_generic method or update_fieldName mehthods for fields with custom but generally applicable logic
   * propvalue
     * for making so called metastatemnt or property statment (old A0093 has relation)
     * it sits in the entity "props" attribute, it has IProp class
   * special
     * fully custom method for the parsing behavior
     *

### Parsing process
 1. Prepare data model entities classes (from typescript inkvisitor classes -> json schema ->  python classes)
 2. Load and wrangle all input data
 3. Process headers of the tables for parsing instructions
 4. Process tables row by row, column by column
 5. Save as json file (which can be imported to the inkVisitor RethinkDB)


# Dev updates: April

## Concepts
  * DONE wordnet_synset_id
    * wordnet_id bounded to resource object R0067

## Texts
 * DONE special_creation_event_id
   * instructions:
     * Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to any of those Es the metaprop "(has) - C0565 "class" - C2642 "creation" (to instantiate the event to its event type = event class).
   * creation_event_label in standalone field
     * this will trigger the operation
   * lot of proptype and propvalue with non-standard schema: new branch of interpreting

## Manuscripts
 * special_repository_label
   * i:
     * For each non-empty, non-NA, non-NS row: (1) generate L entity with label = value in this col., status = "approved", entity logical type = "definite", label language = value in the next col. (repository_label_language); (2) append to this L entity a metaprop (has) - C0565 "class" - C2646 "archive or library", and (3) under the O entity representing the row (i.e. the physical manuscript), add a metaprop which will relate this O to this L entity (repository) through the relation: O - (has) - C2645 "repository" - L in this col.
   * repository_label_language
     * i:
       * Use this value as label language value of the repository L entity.
 * DONE creation_event_id
   * creation_event_label
   * i:
     * Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to any of those Es the metaprop "(has) - C0565 "class" - C2642 "creation" (to instantiate the event to its event type = event class).
 * reproduction_online_url
   * instructions:
     * If non-empty, non-NA, (1) generate an R entity with label "Reproduction of " + label of the MS (i.e. value in the B column, status = "approved", label-language = "English", url = the URL sitting under the hyperlink value in this cell, and (2) add metaprop to the O entity represented by this row: O - (has) - C1199 "digital reproduction" - the R entity here generated.

### Input variables

In [1]:
input_tables = ["texts", "manuscripts", "resources", "actions" , "concepts"]

#                   sheet_name,  code, header_in_row
input_sheets = {
    "texts" : ("Texts","13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5), #https://docs.google.com/spreadsheets/d/13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE/edit#gid=2056508047
    "manuscripts" : ("Manuscripts", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "resources" : ("Resources", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "actions" :  ("Statements","1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ", 5), # https://docs.google.com/spreadsheets/d/1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ/edit#gid=0
    "concepts" : ("Concepts","1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4", 5) # https://docs.google.com/spreadsheets/d/1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4/edit#gid=0
}

table_to_entity = {
    "concepts" : "IConcept",
    "resources" : "IResource",
    "texts" : "ITerritory",
    "manuscripts" : "IObject",
    "actions" :  "IAction",
}

root_sheet_url = "https://docs.google.com/spreadsheets/d/"
google_api_dotenv_path = "../env/.env.googleapi"  # contains google api specs for sheet access with Dator
schema_path = '../schemas/' # path for dir with schemas
json_schemas = {}  # holder for schemas, so they can be used for jsonschema validate

In [2]:
# import subprocess
# subprocess.run("python generate-json-schemas.py", shell=True,capture_output=True)

### Libraries

In [3]:
import os, warlock, json
from datetime import datetime
from jsonschema import validate
import dissinetpytools.dator as dator
from dotenv import load_dotenv

import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import numpy as np
from copy import deepcopy
from shutil import copyfile

import uuid

def get_uuid_id():
    return str(uuid.uuid4())

def is_valid_uuid(val):
    try:
        uuid.UUID(str(val))
        return True
    except ValueError:
        return False


# type hinting
from collections.abc import Sequence, Callable
from typing import List, Dict, Tuple


### Initialisation

In [4]:
load_dotenv(google_api_dotenv_path) # fills os.environ['GDRIVE_API_CREDENTIALS']
d = dator.Dator(loglevel=10, print_log_online=True, cache=True, project_name="inkvisitor-import") # expects 'GDRIVE_API_CREDENTIALS' in the global system variables (os.environ)
d.google_authenticate()
logger = d.logger

20 2022-04-02 21:46:50 : Google authentification start
20 2022-04-02 21:46:50 : Google authentification end
20 2022-04-02 21:46:50 : Dator initiation succesfull end


In [5]:
# read all schemas inside and warlock them as globally available classes
schema_filenames = os.listdir(schema_path)
json_classes = {}
for schema in schema_filenames:
    name = schema.split(".")[0]
    file_handler = open(schema_path + schema,"r")
    schema_json = json.load(file_handler)
    json_schemas[name] = schema_json
    globals()[name] = warlock.model_factory(schema_json)
    json_classes[name] = globals()[name]
    logger.info("Class " + name + " available.")

logger.info(f"There are {len(json_classes.keys())} json classes available ({' '.join(json_classes.keys())}).")

2022-04-02 21:46:50,854 INFO Class IActant available.
2022-04-02 21:46:50,865 INFO Class IAction available.
2022-04-02 21:46:50,874 INFO Class IConcept available.
2022-04-02 21:46:50,885 INFO Class IEntity available.
2022-04-02 21:46:50,896 INFO Class IEvent available.
2022-04-02 21:46:50,903 INFO Class ILabel available.
2022-04-02 21:46:50,912 INFO Class ILocation available.
2022-04-02 21:46:50,921 INFO Class IObject available.
2022-04-02 21:46:50,930 INFO Class IProp available.
2022-04-02 21:46:50,937 INFO Class IReference available.
2022-04-02 21:46:50,948 INFO Class IResource available.
2022-04-02 21:46:50,960 INFO Class IStatement available.
2022-04-02 21:46:50,970 INFO Class ITerritory available.
2022-04-02 21:46:50,980 INFO Class IUser available.
2022-04-02 21:46:50,990 INFO Class IValue available.
2022-04-02 21:46:50,992 INFO There are 15 json classes available (IActant IAction IConcept IEntity IEvent ILabel ILocation IObject IProp IReference IResource IStatement ITerritory IUs

In [133]:
# factory for making entity objects, contains defaults with "prerequisities"
from datetime import datetime
import_note = "Import batch [development] " + str(datetime.now())

class InkVisitorJSONObjectFactory:

    classes = json_classes

    json_class_defaults = {
        'IAction':{
            'class':'A', 'id':'','legacyId':'', 'label':'', 'language':'', 'detail':'','data':{'entities':{'a1':[],'a2':[],'s':[]},'valencies':{'a1':'','a2':'','s':''}}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IConcept':{
            'class':'C', 'id':'', 'legacyId':'','label':'', 'language':'', 'detail':'','data':{}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IValue':{
            'class':'V', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'4'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IProp':{
            'bundleEnd':False,'bundleStart':False, 'certainty':'1', 'children':[], 'elvl':'1',  'id':'', 'logic':'1', 'mood':[], 'moodvariant':'1', 'bundleOperator':'=', 'type': {'id':'','elvl':'1','logic':'1','partitivity':'1','virtuality':'1'},'value':{'id':'','elvl':'1', 'logic':'1', 'partitivity':'1', 'virtuality':'1'}
        },
        'IResource':{
             'class':'R', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'url':'','partValueBaseURL':'','partValueLabel':''}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IObject':{
             'class':'O', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IStatement':{
             'class':'S', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'actants':[], 'actions':[], 'tags':[],'territory': {'id':'','order':0}, 'text':''}, 'props':[], 'notes':[], 'status':'1','references':[]
        },
        'ITerritory':{
            'class':'T', 'id':'', 'legacyId':'','label':'', 'language':'', 'detail':'','data':{'parent':{ "id": "T0", "order": 0 }}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'ILocation':{
            'class':'L', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IEvent':{
            'class':'E', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IReference':{
           'id':'','resource':'','value':''
        }
    }

    def __init__(self):
        for key, item in type(self).json_class_defaults.items():
            if 'notes' in item and  len(item['notes']) == 0:
                item['notes'].append(import_note)

    def make(self, entity_name, override_object=None):
        if override_object is None:
            override_object = {}
        object = type(self).json_class_defaults[entity_name]
        object.update(override_object)
        return type(self).classes[entity_name](deepcopy(object))

    def validate_defaults(self):
        for e in self.json_class_defaults:
            test = self.make(e, self.json_class_defaults[e])
            d.logger.info(f"Class {e} validated.")


IOF = InkVisitorJSONObjectFactory()
IOF.validate_defaults()

2022-04-03 10:35:54,058 INFO Class IAction validated.
2022-04-03 10:35:54,064 INFO Class IConcept validated.
2022-04-03 10:35:54,068 INFO Class IValue validated.
2022-04-03 10:35:54,071 INFO Class IProp validated.
2022-04-03 10:35:54,075 INFO Class IResource validated.
2022-04-03 10:35:54,079 INFO Class IObject validated.
2022-04-03 10:35:54,085 INFO Class IStatement validated.
2022-04-03 10:35:54,090 INFO Class ITerritory validated.
2022-04-03 10:35:54,094 INFO Class ILocation validated.
2022-04-03 10:35:54,099 INFO Class IEvent validated.
2022-04-03 10:35:54,100 INFO Class IReference validated.


# Load input datasets

In [137]:
# empty value unifier
def unify_empty_value(df: pd.DataFrame, empty_values=None, unified_empty_value =''):
    if empty_values is None:
        empty_values = ['NA', "#N/A", "#VALUE!"]
    for naner in empty_values:
        df = df.replace(naner,unified_empty_value)
    df. fillna(unified_empty_value, inplace=True)
    return df



# load all input tables
tables = {}
header_infos = {}
entity_ids = {}
for key, sheet in input_sheets.items():
    logger.info(f"Calling for {key} with sheet_name {sheet[0]}.")
    tables[key], header_infos[key] = d.load_df_from_gsheet(sheet[0],root_sheet_url + sheet[1], sheet[0], fromCache=True, header_in_row=sheet[2], clean=True, fillna=True, cleanByColumn="label", parse_hyperlink_formulas=True, numerize=False)
    tables[key] = unify_empty_value(tables[key])
    header_infos[key] = unify_empty_value(header_infos[key])

    # code for legacyId copy and uuid creation
    tables[key]['legacyId'] = tables[key]['id'].copy()
    # inform instructive header about the new column and what to do with it
    header_infos[key]['legacyId'] = ""
    header_infos[key].at[3,'legacyId'] = "inside"
    tables[key]['id'] = tables[key].apply(lambda x: get_uuid_id(), axis=1)  # generate unique id for each row
    # make id dictionaries (it is much faster to search for keys there in legacyId>id retrievals)
    ed = tables[key][["legacyId","id"]].set_index("legacyId")
    entity_ids[key] = ed["id"].to_dict()

2022-04-03 10:36:34,298 INFO Calling for texts with sheet_name Texts.


20 2022-04-03 10:36:34 : Loading dataset Texts
20 2022-04-03 10:36:34 : Opting for variant header at row 5.
20 2022-04-03 10:36:41 : Hyperlinks were detected and transformed in columns ['edition_1', 'edition_2', 'edition_3', 'persons_index_link', 'places_index_link'].
20 2022-04-03 10:36:41 : Dropping empty columns in the dataset Texts : (1011, 115)


2022-04-03 10:36:41,900 INFO Calling for manuscripts with sheet_name Manuscripts.


20 2022-04-03 10:36:41 : Deleted 869 empty rows by label.
20 2022-04-03 10:36:41 : Loaded and prepared dataset Texts : (142, 115)
20 2022-04-03 10:36:41 : Making pickle cache of  Texts with separeted header file : (142, 115)
20 2022-04-03 10:36:42 : Loading dataset Manuscripts
20 2022-04-03 10:36:42 : Opting for variant header at row 5.


2022-04-03 10:36:46,178 INFO Calling for resources with sheet_name Resources.


20 2022-04-03 10:36:46 : Hyperlinks were detected and transformed in columns ['reproduction_online_url', 'reproduction_note'].
20 2022-04-03 10:36:46 : Dropping empty columns in the dataset Manuscripts : (999, 64)
20 2022-04-03 10:36:46 : Deleted 860 empty rows by label.
20 2022-04-03 10:36:46 : Loaded and prepared dataset Manuscripts : (139, 64)
20 2022-04-03 10:36:46 : Making pickle cache of  Manuscripts with separeted header file : (139, 64)
20 2022-04-03 10:36:46 : Loading dataset Resources
20 2022-04-03 10:36:46 : Opting for variant header at row 5.


2022-04-03 10:36:49,838 INFO Calling for actions with sheet_name Statements.


20 2022-04-03 10:36:49 : Hyperlinks were detected and transformed in columns [].
20 2022-04-03 10:36:49 : Dropping empty columns in the dataset Resources : (1000, 21)
20 2022-04-03 10:36:49 : Deleted 933 empty rows by label.
20 2022-04-03 10:36:49 : Loaded and prepared dataset Resources : (67, 21)
20 2022-04-03 10:36:49 : Making pickle cache of  Resources with separeted header file : (67, 21)
20 2022-04-03 10:36:51 : Loading dataset Statements
20 2022-04-03 10:36:51 : Opting for variant header at row 5.
20 2022-04-03 10:37:08 : Hyperlinks were detected and transformed in columns [].
20 2022-04-03 10:37:08 : Dropping empty columns in the dataset Statements : (1030, 77)
20 2022-04-03 10:37:08 : Deleted 588 empty rows by label.
20 2022-04-03 10:37:08 : Loaded and prepared dataset Statements : (442, 77)
20 2022-04-03 10:37:08 : Making pickle cache of  Statements with separeted header file : (442, 77)


2022-04-03 10:37:08,365 INFO Calling for concepts with sheet_name Concepts.


20 2022-04-03 10:37:08 : Loading dataset Concepts
20 2022-04-03 10:37:08 : Opting for variant header at row 5.
20 2022-04-03 10:37:16 : Hyperlinks were detected and transformed in columns [].
20 2022-04-03 10:37:16 : Dropping empty columns in the dataset Concepts : (3030, 66)
20 2022-04-03 10:37:16 : Deleted 278 empty rows by label.
20 2022-04-03 10:37:16 : Loaded and prepared dataset Concepts : (2752, 66)
20 2022-04-03 10:37:16 : Making pickle cache of  Concepts with separeted header file : (2752, 66)


In [138]:
# making empty table of values
tables['values'] = pd.DataFrame(columns=['id','value','origin'])
tables['locations'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['events'] = pd.DataFrame(columns=['id','value','origin','legacyId'])

In [139]:
header_infos['texts']

,id,label,label_language,label_short,text_name_original,detail,region_covered,region_covered_id,region_covered_label,microregion_covered,...,dissinet_person,number_defendants,number_persons,persons_index_link,places_index_link,old_genre_general,old_genre_label,note,parsing_rows_explained,legacyId
0,,,,,Now concatenated with detail. Creating a new T...,,,,,,...,,,,,,,,,Comment - sometimes important to read for pars...,
1,,,,,,,,,,,...,,,,,,,,,Source node (for props),
2,,,,,,,,,,,...,,,,,,,,note,Prop type (for props) or further detail (for i...,
3,inside,inside,inside,discard,discard,inside,discard,discard,discard,discard,...,discard,discard,discard,discard,discard,discard,discard,inside,discard,inside


In [140]:
tables['actions']

,status,id,label,detail,label_language,subject_entity_type,subject_valency,subject_semantic_id,subject_semantic_label,actant1_entity_type,...,rule_compliance_level,text_example,recommendation,note,editor,parsing_rows_explained,DEPRECATED_label_with_valency_and_notes,DEPRECATED_label_english_with_valency,DEPRECATED_action_or_relation_synonyms,legacyId
0,approved,b3d223b5-5cf3-4f7e-8ec8-3f4a291355fa,abiurationem recepit,received abjuration (from sb),Latin,P | G,1,C0146,inquisitor,P | G,...,,,,,,,abiurationem recepit,received abjuration (from sb - [concerning st]),,A0168
1,approved,7955d11f-aea5-46cb-8066-64bbea2275d9,abiuravit,abjured (in front of sb - [st]),Latin,P | G,1,C0693,reus,P | G,...,,,,,,,abiuravit,abjured (in front of sb - [st]),,A0018
2,pending,7071a447-0761-4ca6-8f5d-905102879c13,abluit,washed away (st),Latin,P | G,1,C1502,cleaner,C | S | O,...,,,,,,,,,,A0428
3,pending,1d07011c-9c3e-4758-be3d-882ff4cdd2f7,abscondit,hid (absol.),Latin,P | G,1,C1433,concealer,NULL,...,O,,,,,,absconduit,hid,,A0366
4,approved,c041da78-db8d-482e-974f-03ad28c4b8ab,absolutus/a,was absolved (by sb - [from a punishment]),Latin,P | G,1,C1447,defendant,P | G,...,,,,,,,absolutus/a fuit,was absolved (by sb - [from some punishment]),,A0214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,approved,b87ab4a8-ae9c-49f8-9218-ae9b2f35126d,vocavit,summoned (sb - [concerning st]),Latin,P | G,1,C1436,summoner,P | G,...,,,,,,,vocavit,summoned (sb - [concerning st]),,A0137
438,pending,c4237a06-17e8-4fef-863d-346f1f59ccf4,vocavit,called [to come] (sb - [concerning st]),Latin,P | G,1,C0994 #C0428,human interactant #inviter,P | G,...,O,,,,,,vocavit (alqm - [de alqo]),called [to come] (sb - [concerning st]),,A0404
439,approved,4836d2cc-1563-4c70-be07-66fff634b1d7,voluit,wanted (st),Latin,P | G,1,C1707,one who wishes something,S | C,...,,,,,,,voluit,wanted (st),,A0145
440,approved,6cb981a7-7061-41a5-a87d-128e7133b7cc,vovit,made a vow (to sb - [concerning st]),Latin,P | G,1,C1648,participant in a socialy normative action,P | G,...,,"""de quibusdam dominabus que voverunt illi sanc...",,,,,vovit,made a vow (to sb - [concerning st]),,A0202


# Declaration of controlling classes

In [150]:
additional_entities = []

# for controlling entity and mapping of its fields
class EntityMapper:

    # simple inside values mapping from input_values in gsheets to inkVisitor enums
    # field: { FROM  : TO }
    enum_mapper = {'language': {"English":"eng","Latin":"lat","Occitan":"oci","Middle English":"enm","Czech":"ces","Italian":"ita","French":"fra","German":"deu"},"status":{"approved":"1","pending":"0","discouraged":"2","warning":"3"}}
    # status  Pending = "0",   Approved = "1",  Discouraged = "2",  Warning = "3",
    valid_entity_classes = ['A','C','E','O','R','T','P','G','S','L','NULL']


    IOF = InkVisitorJSONObjectFactory()

    def __init__(self, entity_type, data_row, logger = d.logger):
        self.entity =  type(self).IOF.make(entity_type)
        self.logger = logger
        self.debug = True
        self.data_row = data_row


    def make_prop_object(self,prop_type_id, prop_value_id):
        prop_object = IOF.make('IProp')
        prop_object['id'] = get_uuid_id()
        prop_object['type']['id'] = prop_type_id
        prop_object['value']['id'] = prop_value_id
        return prop_object

    def make_ref_object(self, ref_resource_id, ref_value_id):
        ref_object = IOF.make('IReference')
        ref_object['id'] = get_uuid_id()
        ref_object['resource'] = ref_resource_id
        ref_object['value'] = ref_value_id
        return ref_object

    def get_entity_id(self,entity_string, origin = ""):
        id = ""
        # logger.info(f"Getting entity string {entity_string} in {origin}")
        try:
            if entity_string.startswith("C") and entity_string[1:].isnumeric():
                # id = tables['concepts'].loc[tables['concepts']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["concepts"][entity_string]
            elif entity_string.startswith("~V~"):
                ventity = self.make_ventity(entity_string, origin=origin)
                id = ventity['id']
            elif entity_string.startswith("M") and entity_string[1:].isnumeric():
                # id = tables['manuscripts'].loc[tables['manuscripts']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["manuscripts"][entity_string]
            elif entity_string.startswith("A") and entity_string[1:].isnumeric():
                # id = tables['actions'].loc[tables['actions']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["actions"][entity_string]
            elif entity_string.startswith("R") and entity_string[1:].isnumeric():
                # id = tables['resources'].loc[tables['resources']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["resources"][entity_string]
            elif entity_string.startswith("T") and entity_string[1:].isnumeric():
                # id = tables['resources'].loc[tables['resources']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["texts"][entity_string]

            elif is_valid_uuid(entity_string):
                id = entity_string

        except IndexError as E:
            logger.info(f"Cannot get entity id from entity string {entity_string} in {origin}. {E}")

        if id != "" and isinstance(id, str):
            # logger.info(f"Got entity id {id} from entity string {entity_string} in {origin}")
            return id
        else:
            logger.error(f"Cannot get entity id from {entity_string}.")
            raise Exception(f"Cannot get entity id from {entity_string}.")

    def make_ventity(self, value_string, origin=""):
        # logger.info(f"Generating ventity from {value_string}.")
        # generate value entity object...
        ventity = IOF.make('IValue')
        ventity['id'] = get_uuid_id()
        ventity['label'] = value_string.replace("~V~","")

        if self.debug:
            ventity['notes'].append(origin)

        # register ventity
        tables['values'] = tables['values'].append({'id':ventity['id'] ,'value':ventity['label'],"origin":origin},ignore_index=True )
        additional_entities.append(ventity)

        # logger.info(f"Ventity id={ventity['id']} generated.")

        return ventity

    def make_rentity(self, label, url = "", origin=""):
        # logger.info(f"Generating rentity from {value_string}.")
        # generate resource entity object...
        rentity = IOF.make('IResource')
        rentity['id'] = get_uuid_id()
        rentity['label'] = label
        rentity['data']['link'] = url

        if self.debug:
            rentity['notes'].append(origin)

        # register rentity
        tables['resources'] = tables['resources'].append({'id':rentity['id'] ,'value':rentity['label'],"origin":origin}, ignore_index=True )
        additional_entities.append(rentity)

        # logger.info(f"Rentity id={rentity['id']} generated.")
        return rentity


    def make_eentity(self, label, legacyId, url = "", origin=""):

        eentity = IOF.make('IEvent')
        eentity['id'] = get_uuid_id()
        eentity['label'] = label
        eentity['legacyId'] = legacyId

        if self.debug:
            eentity['notes'].append(origin)

        # register event
        tables['events'] = tables['events'].append({'id':eentity['id'] ,'value':eentity['label'],"origin":origin, "legacyId":legacyId}, ignore_index=True )
        additional_entities.append(eentity)

        return eentity

    def make_lentity(self, label, legacyId="", url = "", origin=""):

        lentity = IOF.make('ILocation')
        lentity['id'] = get_uuid_id()
        lentity['label'] = label
        lentity['legacyId'] = legacyId

        if self.debug:
            lentity['notes'].append(origin)

        # register lentity
        tables['locations'] = tables['locations'].append({'id':lentity['id'] ,'value':lentity['label'],"origin":origin,"legacyId":legacyId}, ignore_index=True )
        additional_entities.append(lentity)

        return lentity

    # interprets prop_type (should be always concept or resource) and input_value (should be concept or value string)
    # get ids of the prop_type and prop_value (possibly creates and register values object)
    # make iProp object
    # puts iProp object into the entity props property
    def hook_prop_object(self, prop_type, input_value, prop_source="",  origin=""):

        # allowed entities in type
        assert "C" in prop_type[0] or "R" in prop_type[0], f"Prop type unknown, C or R string entity expected? {prop_type}, {origin}"

        # allowed entities in input
        allowed_strict_entities = ['C','M','A','E','L','R','S','V','O','T'] # should be followed by numbers
        allowed_free_string_entities = ['~V~']

        # checking input_value
        if not is_valid_uuid(input_value):
            if any(input_value.startswith(c)for c in allowed_strict_entities):
                # check for numbers
                if not input_value[1:].isnumeric():
                    input_value = "~V~"+input_value
            elif not any(input_value.startswith(c)for c in allowed_free_string_entities):
                input_value = "~V~"+input_value

        # TODO this is useless now, because the line above will make ~V~ from everything uknown
        assert any(input_value.startswith(c)  for c in allowed_strict_entities) or any(input_value.startswith(c)  for c in allowed_free_string_entities) or is_valid_uuid(input_value), f"Prop value unknown, C string or V string entity expected, or valid uuid.{input_value}, {origin}"

        prop_type_id = self.get_entity_id(prop_type, origin=origin)
        if not is_valid_uuid(input_value):
            prop_value_id = self.get_entity_id(input_value, origin=origin)
        else:
            prop_value_id = input_value
        # logger.info(f"{prop_type_id}, {prop_value_id}")

        # make IProp object
        prop_object = self.make_prop_object(prop_type_id, prop_value_id)

        if prop_source !="": # means propvalue_2nd
            self.hook_2ndprop_into_props(prop_object,prop_source)
            pass
        else:
            # hook directly into the entity object
            self.hook_prop_into_props(prop_object)

    def hook_ref_object(self, ref_legacyID, input_value, prop_source="",  origin=""):

        # allowed entities in ref_legacyID
        assert "R" in ref_legacyID, f"Unknown input, R legacyId expected? {ref_legacyID}, {origin}"

        #modify value, so the value object is created
        input_value = "~V~"+input_value

        ref_resource_id = self.get_entity_id(ref_legacyID, origin=origin)
        ref_value_id = self.get_entity_id(input_value, origin=origin)

        # make IReference object
        ref_object = self.make_ref_object(ref_resource_id, ref_value_id)

        self.hook_ref_into_refs(ref_object)


    def hook_prop_into_props(self,prop_object):
        self.entity['props'].append(prop_object)

    def hook_ref_into_refs(self,ref_object):
        self.entity['references'].append(ref_object)

    def hook_2ndprop_into_props(self,prop_object,fst_prop_identification):  # identification by concept id
        key = 0
        keyId = self.get_entity_id(fst_prop_identification)
        assert len(keyId)>0, f"Cannot recognize entity id from {fst_prop_identification}"

        # count, value in enumerate(values)
        for count, po in enumerate(self.entity['props']):
            if po['type']['id'] == keyId and len(po['children']) == 0:  # I am counting on the fact, that if there relations from multiples, they are processed in the specific right order
               po['children'].append(prop_object)

    # method invoker for the INSIDE operation with concrete fields
    def update_inside_field(self, field_name, input_value, origin= ""):
        if input_value != '':

            if ("#" in input_value or "~" in input_value) and field_name!= "note" and "https://docs." not in input_value:
                self.logger.info(f"ALERT # or ~ in the input value {input_value}")

            update_operation = "update_" + field_name
            update_func = getattr(self, update_operation, self.update_generic)
            update_func(field_name, input_value, origin)
        else:
            raise Exception(f"Trying to update {field_name} with empty input value.")

    #########################################################################################################
    # the naming of procedures corresponds to the name of the input_table fields,  used for inside operations

    def update_label_language(self, field_name, input_value, origin = ""):
        if input_value in type(self).enum_mapper['language']:
            self.entity['language'] = type(self).enum_mapper['language'][input_value]
        else:
            self.logger.error(f"Unable to set language in {origin}.")
            self.entity['language'] = input_value # will raise error

    def update_status(self, field_name, input_value, origin = ""):
        if input_value in type(self).enum_mapper['status']:
            self.entity['status'] = type(self).enum_mapper['status'][input_value]
        else:
            self.logger.error(f"Unable to set status in {origin}.")
            self.entity['status'] = input_value # will raise error

    def update_note(self, field_name, input_value, origin = ""):
        #self.logger.info(f"Updating note with {input_value}.")
        if "#" in input_value:
            values = [v.strip() for v in input_value.split("#")]
            for v in values:
                self.entity['notes'].append(v)
        else:
            self.entity['notes'].append(input_value)

    def update_id(self, field_name, input_value, origin = ""):
        # self.entity['id'] = input_value
        self.entity['id'] = input_value

    def update_legacyId(self, field_name, input_value, origin = ""):
        # logger.info(f"Trying to set legacyId {type(input_value)}:'{input_value}' {origin}.")
        self.entity['legacyId'] = input_value

    def update_label(self, field_name, input_value, origin = ""):
        self.entity['label'] = input_value

    def update_wordnet_lemma_id(self, field_name, input_value, origin = ""):
        # self.logger.info(f" wordnet_lemma_id NOT IMPLEMENTED ")
        pass

    def update_wordnet_synset_id(self, field_name, input_value, origin = ""):
        # self.logger.info(f" wordnet_synset_id NOT IMPLEMENTED ")
        pass

    def update_generic(self, field_name, input_value, origin = ""):
        self.entity[field_name] = input_value



class TerritoryEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class ConceptEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class ActionEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger,):
        EntityMapper.__init__(self,entity_type, data_row, logger)

    def update_subject_entity_type(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: subject_entity_type")
        entities = [e.strip() for e in value.split("|")]
        for e in entities:
            if e in self.valid_entity_classes:
                self.entity['data']['entities']['s'].append(e)
            elif e == "*":
                self.entity['data']['entities']['s'] = self.valid_entity_classes
            else:
                logger.error(f"Non valid entity processed: {e} while AP.update_subject_entity_type().")

    def update_subject_valency(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: subject_valency")
        self.entity['data']['valencies']['s'] = value

    def update_actant1_entity_type(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant1_entity_type")
        entities = [e.strip() for e in value.split("|")]
        for e in entities:
            if e in self.valid_entity_classes:
                self.entity['data']['entities']['a1'].append(e)
            elif e == "*":
                self.entity['data']['entities']['a1'] = self.valid_entity_classes
            else:
                logger.error(f"Non valid entity processed: {e} while AP.update_actant1_entity_type().")

    def update_actant2_entity_type(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant2_entity_type")
        entities = [e.strip() for e in value.split("|")]
        for e in entities:
            if e in self.valid_entity_classes:
                self.entity['data']['entities']['a2'].append(e)
            elif e == "*":
                self.entity['data']['entities']['a2'] = self.valid_entity_classes
            else:
                logger.error(f"Non valid entity processed: {e} while AP.update_actant2_entity_type().")

    def update_actant1_valency(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant1_valency")
        self.entity['data']['valencies']['a1'] = value

    def update_actant2_valency(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant1_valency")
        self.entity['data']['valencies']['a2'] = value


class ResourceEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

    def update_url(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: subject_entity_type")
        self.entity['data']['url'] = value


class ObjectEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class EventEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class LocationEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)


class EntityMapperFactory:
    def __init__(self):
        pass

    def make(self, entity_name, data_row):
        if 'ITerritory' == entity_name:
            return TerritoryEntityMapper(entity_name, data_row)
        elif 'IAction' == entity_name:
            return ActionEntityMapper(entity_name, data_row)
        elif 'IConcept' == entity_name:
            return ConceptEntityMapper(entity_name,data_row)
        elif 'IResource' == entity_name:
            return ResourceEntityMapper(entity_name, data_row)
        elif 'IObject' == entity_name:
            return ObjectEntityMapper(entity_name, data_row)
        elif 'IEvent' == entity_name:
            return EventEntityMapper(entity_name, data_row)
        elif 'ILocation' == entity_name:
            return LocationEntityMapper(entity_name, data_row)

        else:
            logger.warning(f"Unrecognized entity class in entity mapper. Is this right?")
            return EntityMapper(entity_name)


# CONTROL CLASS
class ParseController():

    def __init__(self, entity_list: [], keyword_row_id = 3,  logger = d.logger):
        self.entity_list = entity_list
        self.logger = logger
        self.parsers = {}
        self.js_objects = []

        for e in self.entity_list:
            if 'texts' in e:
                self.parsers[e] = TextParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'actions' in e:
                self.parsers[e] = ActionParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'concepts' in e:
                self.parsers[e] = ConceptParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'resources' in e:
                self.parsers[e] = ResourceParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'manuscripts' in e:
                self.parsers[e] = ManuscriptParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            else:
                self.logger.warning(f"Coming to basic Parser entity - strange '{e}' {type(e)}.")
                self.parsers[e] = Parser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)

    def load_json_objects(self):
        for name, p in self.parsers.items():
            self.js_objects = self.js_objects + p.js_objects

    def parse(self):
        for name, p in self.parsers.items():
            p.parse_rows()


# WORKER CLASS
class Parser():
    EMP = EntityMapperFactory()

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        self.name = name
        self.logname = name.upper()
        self.input_header_df = header_df
        self.input_table_df = table_df
        self.prepared_table = pd.DataFrame()
        self.keyword_row_id =  keyword_row_id
        self.columns = self.input_header_df.columns.tolist()

        self.parsing_instruction = {}
        self.oper_columns = {'discard':[],'inside':[],'special':[],'unknown':[],'propvalue':[],'propvalue_2nd':[],"dependent":[],"proptype":[]}
        self.logger = logger

        # parsed json data holder
        self.js_objects = []

        # RUN
        self.process_header_instructions()
        self.prepare_input_table()

    # "parsing" instructions
    def process_header_instructions(self) -> (pd.DataFrame, pd.DataFrame):
        keyword_row = self.input_header_df.iloc[self.keyword_row_id]
        prop_type_row = self.input_header_df.iloc[self.keyword_row_id - 1]
        source_node_row = self.input_header_df.iloc[self.keyword_row_id - 2]

        log_uncertain_instructions = []

        for c in self.columns:
            instruction_candidate = str(keyword_row.at[c]).strip()
            prop_type_candidate = str(prop_type_row.at[c]).strip()
            source_node_candidate = str(source_node_row.at[c]).strip()

            if c == '':
                self.logger.error(f"{self.logname} There is empty column in the dataset.")
                raise Exception(f"{self.logname} There is empty column in the dataset.")

            if "?" in instruction_candidate or "?" in prop_type_candidate or "?" in source_node_candidate:
                log_uncertain_instructions.append(f"{c.upper()}:{instruction_candidate},{prop_type_candidate},{source_node_candidate}")
                instruction  = {'operation':'discard', 'target': None}
                self.oper_columns['discard'].append(c)


            # known instructions
            if 'discard' in instruction_candidate:
                instruction  = {'operation':'discard', 'target': None}
                self.oper_columns['discard'].append(c)

            elif 'propvalue' ==  instruction_candidate:
                prop_type = prop_type_candidate
                source_node = source_node_candidate

                # test whether is its propvalue proper or dependent (=proptype is dynamic, value from another column)
                if prop_type.strip() == "":
                    self.oper_columns['dependent'].append(c)
                    continue # ignoring "dependent propvalue"

                if "?" in prop_type or "?"  in source_node:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'propvalue', 'type': prop_type, 'source':source_node} # source can be ignored, because the iProp object is sitting inside of it
                    self.oper_columns['propvalue'].append(c)

            elif 'propvalue_2nd' in instruction_candidate:
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                if "?" in prop_type or "?"  in source_node:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'propvalue_2nd', 'type': prop_type, 'source':source_node} # source can NOT be ignored, it signals which existing iProp object will hold this iProp object
                    self.oper_columns['propvalue_2nd'].append(c)

            elif 'special' in instruction_candidate:
                # looks for custom functions registered by column name
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                instruction  = {'operation':'special', 'type': prop_type, 'source':source_node}
                self.oper_columns['special'].append(c)

            elif 'proptype' in instruction_candidate:
                instruction  = {'operation':'proptype', 'type': prop_type, 'source':source_node}
                #logger.info(f"here ...{instruction_candidate} {c}")
                self.oper_columns['proptype'].append(c)

            elif 'dependent' in instruction_candidate:
                # ignore
                # the value is solved by another instruction
                instruction  = {'operation':'dependent', 'type': prop_type, 'source':source_node}

            elif 'inside' in instruction_candidate:
                if "?" in c:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'inside', 'target': None}
                    if len(prop_type_candidate) > 0:
                        instruction  = {'operation':'inside', 'target': prop_type_candidate}

                    self.oper_columns['inside'].append(c)

            else:
                instruction = {'operation':'unknown', 'target': None}
                self.oper_columns['unknown'].append(c)
            self.parsing_instruction[c] = instruction

        self.logger.info(f"{self.logname} Uncertain parsing instructions in {len(log_uncertain_instructions)} columns: " + " ".join(log_uncertain_instructions) + ".")
        return self.parsing_instruction

    def prepare_input_table(self):
        ip = self.input_table_df.copy()

        # discard  columns with discard and unknown operations
        ip.drop(columns=self.oper_columns['discard']+self.oper_columns['unknown'], inplace=True)

        self.logger.info(f"{self.logname} {len(self.oper_columns['discard']+self.oper_columns['unknown'])} columns have been dropped (discard:{len(self.oper_columns['discard'])}, unknown:{len(self.oper_columns['unknown'])}). Table now has {len(ip.columns)} columns, inside:{len(self.oper_columns['inside'])},propvalue:{len(self.oper_columns['propvalue'])}, special:{len(self.oper_columns['special'])}, proptype: {len(self.oper_columns['proptype'])}, dependent:{len(self.oper_columns['dependent'])}. Originally {self.input_table_df.shape[1]} columns.")

        self.prepared_table = ip


    def prepare_property(self):
        pass


    def make_row_object(self, data_row):
        class_name = table_to_entity[self.name]
        return self.EMP.make(class_name, data_row)

    def itemize_valuestring_for_multiples(self, value_with_multiples, origin="") -> []:
        values = []
        if isinstance(value_with_multiples,str):
            parsed_value = value_with_multiples.split('#')
            values =  [item.strip() for item in parsed_value]
        else:
            raise Exception(f"Expected value to be string. Got {type(value_with_multiples)}. {origin}")
        return values

    def parse_rows(self):
        self.logger.info(f"Starting to parse {self.name}.")

        if  self.prepared_table['label'].isnull().any():
            self.prepared_table  = self.prepared_table[self.prepared_table['label'].notna()]
            self.logger.info(f"Empty labels found in {self.name} table. (new entities added through parsing process). Adjusting.")

        for key, row in self.prepared_table.iterrows():
            # self.logger.info(f"{self.name} Processing row {key}")
            entity_mapper = self.make_row_object(row.to_dict())

            for name, value in row.items():
                if name in self.parsing_instruction:
                    operation = self.parsing_instruction[name]
                else:
                    continue # silently ignore unknown columns
                # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")

                # force string
                value = str(value)

                if operation['operation'] == 'inside' and value != '' and '?' not in name:
                    # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")
                    if operation['target']:
                        name = operation['target']
                    entity_mapper.update_inside_field(name,value,operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value))

                if operation['operation'] == 'propvalue' and value != '':
                    # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")
                    prop_type = operation['type']
                    if prop_type == '' or 'C' not in prop_type:
                        raise Exception(f"Propvalue does not have prop type defined. C entity-string expected, got {key}, {name}, {value}")
                    for item in self.itemize_valuestring_for_multiples(value):
                        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = item, origin = operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value))

                if operation['operation'] == 'propvalue_2nd' and value != '':
                    # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")
                    prop_type = operation['type']
                    prop_source_name = operation['source']  # header_name,  we need concept_id

                    prop_source_id = self.parsing_instruction[prop_source_name]['type']
                    assert 'C' in prop_source_id, f"Trying to get to the concept of 1st level property to adress 2nd level property."

                    if prop_type == '' or 'C' not in prop_type:
                        raise Exception(f"Propvalue does not have prop type defined. C entity-string expected, got {key}, {name}, {value}")

                    origin = operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value)

                    for item in self.itemize_valuestring_for_multiples(value, origin=origin):
                        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = item, prop_source = prop_source_id, origin = origin)
                    pass

                if operation['operation'] == 'special' and value != "":
                    # logger.info(f"SPECIAL {operation} {value}")
                    func = getattr(self, 'special_'+name)
                    func(operation, value, entity_mapper)

            self.js_objects.append(entity_mapper.entity)
            # break  DEV, checking parsing after first iteration


    def special_editor(self, operation, value, entity_mapper, field_name="special_editor"):
        pass


class TextParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    # special methods for fields, which needs fully individual processing
    def special_edition_1(self, operation, value, entity_mapper, field_name="special_edition1", ):
        # Parse this col. as "propvalue" - but you need to generate the target entities since they do not exist. How to do it: for any value, create an R entity with "label" = textual value in this col., "label language" = English, "status" = "approved", and "URL" = the hyperlink in the formula sitting on the textual value in this col. As usual, ignore NS and NA values (exact match) - do not import anything if the value is NA.
        # logger.info(f"Special edition1 running ...{operation} {value}")

        origin = operation['operation'] +" "+ operation['type']+ ">"+ ":"+field_name + str(value)
        prop_type = operation['type']

        # make rentity
        # logger.info(f"Rentity making {value}")
        if "|" in value:
            data = value.split("|")
            label = data[0]
            url = data[1]
        else:
            url = ""
            label = value
            # logger.warning(f"Expected char | signaling url after label. Got just {value}."+origin)

        rentity = entity_mapper.make_rentity(label, url, origin=origin)

        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = rentity['id'], origin = origin)

    def special_edition_2(self, operation, value, entity_mapper):
        self.special_edition_1( operation, value, entity_mapper, field_name="edition_2")

    def special_edition_3(self, operation, value, entity_mapper):
        self.special_edition_1( operation, value, entity_mapper, field_name="edition_3")

    def special_creation_event_id(self, operation, value, entity_mapper : EntityMapper, field_name="creation_event_id"):
        # Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new “hash” ID from the db, (3) label of this E: see next col., (4) logical type “definite” (default), (5) label language “English”, (6) status “approved”, and (7) attach to any of those Es the metaprop "(has) - C0565 “class” - C2642 “creation” (to instantiate the event to its event type = event class).

        origin = operation['operation'] +">"+ ":"+field_name + str(value)

        data = entity_mapper.data_row
        event_entity = entity_mapper.make_eentity(data['creation_event_label'], legacyId = value, origin = origin)

        prop_type_id = entity_mapper.get_entity_id("C0565")
        prop_value_id = entity_mapper.get_entity_id("C2642")

        # make IProp object
        prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)


        # hook prop object to the event entity
        event_entity['props'].append(prop_object)

        # hook the vent event to the territory
        entity_mapper.hook_prop_object(prop_type = "C2642", input_value = event_entity['id'], origin = origin)

        # process time_relations
        t_relations = [(
            data["timerelation1_type_conceptified_id"], data["timerelation1_target_id"]
        ), (
            data["timerelation2_type_conceptified_id"], data["timerelation2_target_id"]
        ),(
            data["timerelation3_type_conceptified_id"], data["timerelation3_target_id"]
        ), (
            data["timerelation4_type_conceptified_id"], data["timerelation4_target_id"]
        )]

        origin = data['legacyId']
        for o in t_relations:
            if len(o[0]) > 0 and len(o[1]) > 0:
                prop_type_id = entity_mapper.get_entity_id(o[0], origin = origin)
                prop_value_id = entity_mapper.get_entity_id("~V~"+o[1], origin = origin)
                prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
                event_entity['props'].append(prop_object)


    # empty, operation is solved by f above
    def special_creation_event_label(self, operation, value, entity_mapper):
        pass

class ActionParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)


class ConceptParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    def special_wordnet_synset_id(self, operation, value, entity_mapper,field_name="special_wordnet_synset_id",):
        # wordnet_resource_id = R0067
        entity_mapper.hook_ref_object(ref_legacyID = "R0067", input_value = value, origin = operation['operation'] +">"+ ":"+field_name + " " +str(value))


class ManuscriptParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    def special_creation_event_id(self, operation, value, entity_mapper : EntityMapper, field_name="creation_event_id"):
        # Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new “hash” ID from the db, (3) label of this E: see next col., (4) logical type “definite” (default), (5) label language “English”, (6) status “approved”, and (7) attach to any of those Es the metaprop "(has) - C0565 “class” - C2642 “creation” (to instantiate the event to its event type = event class).

        data = entity_mapper.data_row
        event_entity = entity_mapper.make_eentity(data['creation_event_label'], legacyId = value)

        prop_type_id = entity_mapper.get_entity_id("C0565")
        prop_value_id = entity_mapper.get_entity_id("C2642")

        # make IProp object
        prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)


        # hook prop object to the event entity
        event_entity['props'].append(prop_object)

        # hook the vent event to the territory
        entity_mapper.hook_prop_object(prop_type = "C2642", input_value = event_entity['id'], origin = operation['operation'] +">"+ ":"+field_name + str(value))

        # process time_relations
        t_relations = [(
            data["timerelation1_type_conceptified_id"], data["timerelation1_target_id"]
        ), (
            data["timerelation2_type_conceptified_id"], data["timerelation2_target_id"]
        ),(
            data["timerelation3_type_conceptified_id"], data["timerelation3_target_id"]
        ), (
            data["timerelation4_type_conceptified_id"], data["timerelation4_target_id"]
        )]

        origin = data['legacyId']
        for o in t_relations:
            if len(o[0]) > 0 and len(o[1]) > 0:
                prop_type_id = entity_mapper.get_entity_id(o[0], origin = origin)
                prop_value_id = entity_mapper.get_entity_id("~V~"+o[1], origin = origin)
                prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
                event_entity['props'].append(prop_object)


    # empty, operation is solved by f above
    def special_creation_event_label(self, operation, value, entity_mapper):
        pass


    def special_repository_label(self, operation, value, entity_mapper, field_name = "repository_label"):
        # For each non-empty, non-NA, non-NS row: (1) generate L entity with label = value in this col., status = “approved”, entity logical type = “definite”, label language = value in the next col. (repository_label_language); (2) append to this L entity a metaprop (has) - C0565 “class” - C2646 “archive or library”, and (3) under the O entity representing the row (i.e. the physical manuscript), add a metaprop which will relate this O to this L entity (repository) through the relation: O - (has) - C2645 “repository” - L in this col.

        origin = f"Making location '{value}' from " + entity_mapper.data_row['legacyId'] + " by field {field_name}."

        if value != "" and value!="NA" and value!="N/A" and value!="NS":  # check value
            lentity = entity_mapper.make_lentity(label=value, legacyId="L_from_"+entity_mapper.data_row['legacyId'], origin=origin)

            lentity['language'] = entity_mapper.enum_mapper['language'][entity_mapper.data_row['repository_label_language']]

            prop_type_id = entity_mapper.get_entity_id("C0565", origin = origin)
            prop_value_id = entity_mapper.get_entity_id("C2646", origin = origin)
            prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
            lentity['props'].append(prop_object)

            # let the manuscript O own the location
            entity_mapper.hook_prop_object(prop_type = "C2645", input_value = lentity['id'], origin = origin)

    # solves the method above
    def special_repository_label_language(self, operation, value, entity_mapper):
        pass

    def special_reproduction_online_url(self, operation, value, entity_mapper, field_name = "reproduction_online_url"):
        # If non-empty, non-NA, (1) generate an R entity with label "Reproduction of " + label of the MS (i.e. value in the B column, status = “approved”, label-language = “English”, url = the URL sitting under the hyperlink value in this cell, and (2) add metaprop to the O entity represented by this row: O - (has) - C1199 “digital reproduction” - the R entity here generated.

        origin = f"Making Resource entity '{value}' from " + entity_mapper.data_row['legacyId'] + f" by field {field_name}."

        if value != "" and value!="NA" and value!="N/A" and value!="NS":  # check value

            if "|" in value:
                data = value.split("|")
                label = data[0]
                url = data[1]
            else:
                url = ""
                label = value
                logger.warning(f"Expected char | signaling url after label. Got just {value}. "+origin)

            rentity = entity_mapper.make_rentity(label, url, origin=origin)
            entity_mapper.hook_prop_object(prop_type = "C1199", input_value = rentity['id'], origin = origin)


class ResourceParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

# Run

In [151]:
logger.info(f"Start")
cp = ParseController(entity_list=['texts','manuscripts','resources','concepts','actions'])
# cp = ParseController(entity_list=['resources'])
cp.parse()
logger.info(f"End")

2022-04-03 10:53:35,385 INFO Start
2022-04-03 10:53:35,392 INFO TEXTS Uncertain parsing instructions in 0 columns: .
2022-04-03 10:53:35,394 INFO TEXTS 66 columns have been dropped (discard:66, unknown:0). Table now has 50 columns, inside:9,propvalue:14, special:5, proptype: 4, dependent:4. Originally 116 columns.
2022-04-03 10:53:35,398 INFO MANUSCRIPTS Uncertain parsing instructions in 0 columns: .
2022-04-03 10:53:35,400 INFO MANUSCRIPTS 35 columns have been dropped (discard:35, unknown:0). Table now has 30 columns, inside:8,propvalue:9, special:5, proptype: 4, dependent:4. Originally 65 columns.
2022-04-03 10:53:35,402 INFO RESOURCES Uncertain parsing instructions in 0 columns: .
2022-04-03 10:53:35,403 INFO RESOURCES 14 columns have been dropped (discard:14, unknown:0). Table now has 10 columns, inside:6,propvalue:2, special:0, proptype: 0, dependent:0. Originally 24 columns.
2022-04-03 10:53:35,407 INFO CONCEPTS Uncertain parsing instructions in 0 columns: .
2022-04-03 10:53:35,4

### Profiling

In [12]:
#%reload_ext line_profiler

# def profile():
#    cp.parsers['concepts'].parse_rows()

#%lprun -f EntityMapper.get_entity_id profile()

In [13]:
# %prun -s tottime profile()

In [14]:
# %lprun?

In [147]:
tables['events']

,id,value,origin,legacyId
0,5526ffbd-486d-445d-8c7a-4dd15acbe7d8,"Creation of ""Process against Bernard Niort and...",,E0001_T1
1,f6290002-050d-4d72-9f80-4902a5c4a26a,Creation of 'Sentences of William Arnold and S...,,E0002_T2
2,e0522c0a-59e3-46d4-b793-4e4133c34bde,Creation of 'Peter Seila’s Register of Penances',,E0003_T3
3,38528e5f-855c-4457-a149-90c43cac7507,Creation of 'Register FFF of the Carcassonne i...,,E0004_T4
4,4aa68b7d-471f-464f-8319-815536ff0a54,Creation of 'Confirmation of depositions befor...,,E0005_T5
...,...,...,...,...
276,eba5c362-80e6-4cca-a9ea-e6ccfb7c1582,"Creation of manuscript St Paul-im-Lavanttal, S...",,E0135_M153
277,9f4ea920-b74a-4729-9a5d-6210dbd8cbf3,"Creation of manuscript codex Roma, Biblioteca ...",,E0136_M154
278,5568f736-d4b2-442d-a759-79be88723443,"Creation of manuscript codex Roma, Archivio Ge...",,E0137_M155
279,9b5babb2-258d-47a1-87c3-9cdcfea4b0e1,"Creation of manuscript codex Wien, Österreichi...",,E0138_M156


In [124]:
# what is this d72b2bee-47b4-4abc-8c88-793c72ab676d ?

pd.DataFrame(additional_entities)

,class,id,label,language,detail,data,props,notes,status,references,legacyId
0,V,edda2d72-471c-412f-be6d-2d7d38365f16,*,,,{'logicalType': '4'},[],"[Import batch 2022-04-02 23:28:10.114166, Impo...",1,[],NaN
1,V,2a98510b-3238-4ac6-99f0-a5049827a9c3,*,,,{'logicalType': '4'},[],"[Import batch 2022-04-02 23:28:10.114166, Impo...",1,[],NaN
2,V,4aff2a63-30e9-42ca-9907-1a828f8cd059,*,,,{'logicalType': '4'},[],"[Import batch 2022-04-02 23:28:10.114166, Impo...",1,[],NaN
3,V,f0c5d0c1-18fe-4249-b449-2930269da82a,*,,,{'logicalType': '4'},[],"[Import batch 2022-04-02 23:28:10.114166, Impo...",1,[],NaN
4,V,2ba63bec-bc3f-4ee2-b3bf-d20d80e758ef,*,,,{'logicalType': '4'},[],"[Import batch 2022-04-02 23:28:10.114166, Impo...",1,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...
1865,V,1f97529c-ab60-45b7-9ed1-90ad02d6ce08,???,,,{'logicalType': '4'},[],"[Import batch 2022-04-02 23:28:10.114166, Impo...",1,[],NaN
1866,V,9d267bbf-586c-4d86-b86b-6d54292bf9ca,???,,,{'logicalType': '4'},[],"[Import batch 2022-04-02 23:28:10.114166, Impo...",1,[],NaN
1867,V,dba15152-e37d-4e74-9b6e-b1d0c83bbf24,???,,,{'logicalType': '4'},[],"[Import batch 2022-04-02 23:28:10.114166, Impo...",1,[],NaN
1868,V,e5929922-6c80-4fda-a7f1-81183a0c0b33,???,,,{'logicalType': '4'},[],"[Import batch 2022-04-02 23:28:10.114166, Impo...",1,[],NaN


### Last checks

In [153]:
print(json.dumps(cp.parsers['actions'].js_objects[0]))

{"class": "A", "id": "b3d223b5-5cf3-4f7e-8ec8-3f4a291355fa", "legacyId": "A0168", "label": "abiurationem recepit", "language": "lat", "detail": "received abjuration (from sb)", "data": {"entities": {"a1": ["P", "G"], "a2": ["NULL"], "s": ["P", "G"]}, "valencies": {"a1": "\"a\" + 6 | 2", "a2": "NULL", "s": "1"}}, "props": [{"bundleEnd": false, "bundleStart": false, "certainty": "1", "children": [], "elvl": "1", "id": "bd8727c7-23c1-4d3e-8334-c5678e9e1a37", "logic": "1", "mood": [], "moodvariant": "1", "bundleOperator": "=", "type": {"id": "5228b299-d77a-41e1-bbff-c5602eda74d0", "elvl": "1", "logic": "1", "partitivity": "1", "virtuality": "1"}, "value": {"id": "dac22c9c-4731-4da5-8b7b-14fc821fe199", "elvl": "1", "logic": "1", "partitivity": "1", "virtuality": "1"}}, {"bundleEnd": false, "bundleStart": false, "certainty": "1", "children": [], "elvl": "1", "id": "983f87cf-88e7-476e-bb73-899c7c30f351", "logic": "1", "mood": [], "moodvariant": "1", "bundleOperator": "=", "type": {"id": "b2ae

In [152]:
logger.info(f"There are {len(additional_entities)} additionally created entities (e.g. values, resources ...).")

2022-04-03 10:59:31,174 INFO There are 1865 additionally created entities (e.g. values, resources ...).


# Output the parsed data

In [155]:
# root data
root_territory = """
 {
    "id": "T0",
    "class": "T",
    "data": { "parent": false },
    "label": "everything",
    "detail": "",
    "language": "lat",
    "notes": [],
    "props": []
  }
"""

In [ ]:
# connects outcomes from the individual parsers
cp.load_json_objects()

In [156]:
# save json object list of the parsed entities from google sheets in the text file
with open('../datasets/all-test/new_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(cp.js_objects, f)

# save json object list of the "newly created entities" in the text file
with open('../datasets/all-test/additional_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(additional_entities, f)

# read  entities.json
with open('../datasets/all/entities.json','r') as f:
    #entities_content = f.readlines()
    entities_content = f.read().replace('\n', '')

# read  entities.json
with open('../datasets/all-test/new_entities.json','r') as f:
    #entities_content = f.readlines()
    new_entities_content = f.read().replace('\n', '')

additional_entities_string = json.dumps(additional_entities)

# write new and combine with old test data
with open('../datasets/all-test/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content = entities_content[0:-1] +", " + additional_entities_string[1:-1]+ ", " + new_entities_content[1:]
    f.write(str(merge_content))



# write just the new parse data to the entities json.
with open('../datasets/all-parsed/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content =  "[" + root_territory + "," + additional_entities_string[1:-1]+ ", " + new_entities_content[1:]
    f.write(str(merge_content))

logger.info("END")

2022-04-03 11:00:04,572 INFO END
